In [1]:
from SRC.geofunc import *
from SRC.mongofunc import connectMongoColl

from haversine import haversine
# harvesine (lat,long) --> dist in km.

import pandas as pd
from pymongo import MongoClient
import numpy as np

## Not necesary
#from IPython.display import Image
#Install first: from pprint import pprint 


# Generación de posibles localizaciones - cleanspots
---

In [2]:
#Mongo connection
db, collcomp = connectMongoColl('companies','companies')
filt = {"name":1,"offices":1,"category_code":1}
companies = list(collcomp.find({},filt))
df = pd.DataFrame(companies)
display(df.head(8))
print(df.shape)

,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,"[{'description': 'Headquarters', 'address1': '..."
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,"[{'description': '', 'address1': '710 - 2nd Av..."
2,52cdef7c4bab8bd675297d8c,Zoho,software,"[{'description': 'Headquarters', 'address1': '..."
3,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,"[{'description': '', 'address1': 'Suite 200', ..."
4,52cdef7c4bab8bd675297d90,Postini,web,"[{'description': None, 'address1': '959 Skyway..."
5,52cdef7c4bab8bd675297d91,Geni,web,"[{'description': 'Headquarters', 'address1': '..."
6,52cdef7c4bab8bd675297d8d,Digg,news,"[{'description': None, 'address1': '135 Missis..."
7,52cdef7c4bab8bd675297d92,Flektor,games_video,"[{'description': None, 'address1': '8536 Natio..."


(18801, 4)


In [3]:
#HACER UNA FUNCIÓN DE ESTO!

# 1. Hacemos explode para generar una fila por cada elemento de la lista en la columna "offices"
df = df.explode('offices')
display(df.head(6))
print(df.shape)

#2. Hacemos expand de los datos de la columna office, para generar una culumna nueva 
#por cada valor dentro de la lista. axis=1 quiere decir que se recorre fila a fila.
dfOfficeData = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
display(dfOfficeData.head(6))
print(dfOfficeData.shape)

#3. Concatenamos con el df original y limpiamos los datos
cleanData = pd.concat([df,dfOfficeData], axis=1)
cleanData = cleanData.drop(columns=["_id","offices"])
display(cleanData.head(6))
print(cleanData.shape)

,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,"{'description': 'Headquarters', 'address1': '4..."
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '710 - 2nd Ave..."
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '270 Lafayette..."
2,52cdef7c4bab8bd675297d8c,Zoho,software,"{'description': 'Headquarters', 'address1': '4..."
3,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,"{'description': '', 'address1': 'Suite 200', '..."
4,52cdef7c4bab8bd675297d90,Postini,web,"{'description': None, 'address1': '959 Skyway ..."


(21762, 4)


,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
2,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
4,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573


(21762, 9)


,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
2,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
4,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573


(21762, 11)


In [4]:
#Usando la funciónn asGeoJSON: Generamos la columna localización POINT(latitude,longitude)
cleanData["location"] = cleanData[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
cleanData = cleanData.drop(columns=["latitude","longitude"])
display(cleanData.head(8))
print(cleanData.shape)

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location
0,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,"{'type': 'Point', 'coordinates': [-122.333253,..."
1,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,"{'type': 'Point', 'coordinates': [-73.9964312,..."
2,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,..."
3,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,None
4,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,"{'type': 'Point', 'coordinates': [-122.247573,..."
5,Geni,web,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,"{'type': 'Point', 'coordinates': [-118.393064,..."
6,Digg,news,None,135 Mississippi St,,94107,San Francisco,CA,USA,"{'type': 'Point', 'coordinates': [-122.394523,..."


(21762, 10)


In [5]:
cleanData.location.isna().value_counts()

True     10928
False    10834
Name: location, dtype: int64

In [6]:
cleanData = cleanData.dropna(subset=["location"])

In [7]:
cleanData.shape

(10834, 10)

In [8]:
cleanData.head()

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location
0,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,"{'type': 'Point', 'coordinates': [-122.333253,..."
1,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,"{'type': 'Point', 'coordinates': [-73.9964312,..."
2,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,..."
4,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,"{'type': 'Point', 'coordinates': [-122.247573,..."


In [9]:
cleanData.country_code.value_counts()

USA    7600
GBR     611
CAN     449
DEU     280
FRA     255
       ... 
ATG       1
VNM       1
LBN       1
AFG       1
VGB       1
Name: country_code, Length: 79, dtype: int64

In [10]:
#Filtrar los que pertenecen a USA
cleanData = cleanData[cleanData['country_code'].isin(["USA"])]

In [11]:
cleanData.country_code.value_counts()

USA    7600
Name: country_code, dtype: int64

In [12]:
cleanData.to_json("OUTPUT/00CleanSpots.json", orient="records")

In [13]:
#Import to mongodb:
# $ mongoimport --db companies --collection cleanspots --jsonArray --drop OUTPUT/00CleanSpots.json

## FILTER 1 | NOT Near 2km from more than 10 years companies. - oldcompanies
---

In [14]:
db, collcomp = connectMongoColl('companies','companies')
#qfirst = {'founded_year':{'$gte':2010}}
qsecond = {"$and":[{'founded_year':{'$lte':2010}},{"deadpooled_year":None}]}
#qthird = {"$and":[{'founded_year':{'$lte':2010}},{"deadpooled_year":None},{"$or":[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]}]}
filt = {"name":1,"offices":1,"category_code":1}
oldcomp = list(collcomp.find(qsecond,filt))
dfold = pd.DataFrame(oldcomp)
dfold.head()

,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d90,Postini,web,"[{'description': None, 'address1': '959 Skyway..."
1,52cdef7c4bab8bd675297d91,Geni,web,"[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d8d,Digg,news,"[{'description': None, 'address1': '135 Missis..."
3,52cdef7c4bab8bd675297d93,Fox Interactive Media,web,"[{'description': '', 'address1': '407 N Maple ..."
4,52cdef7c4bab8bd675297d96,Gizmoz,web,"[{'description': None, 'address1': None, 'addr..."


In [15]:
# 1. Hacemos explode para generar una fila por cada elemento de la lista en la columna "offices"
dfold = dfold.explode('offices')
#display(dfold.head(6))
#print(dfold.shape)

#2. Hacemos expand de los datos de la columna office, para generar una culumna nueva 
#por cada valor dentro de la lista. axis=1 quiere decir que se recorre fila a fila.
dfOldData = dfold[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
#display(dfOldData.head(6))
#print(dfOldData.shape)

#3. Concatenamos con el df original y limpiamos los datos
cleanOld = pd.concat([dfold,dfOldData], axis=1)
cleanOld = cleanOld.drop(columns=["_id","offices"])
#display(cleanOld.head(6))
#print(cleanOld.shape)

#4. Usando la funciónn asGeoJSON: Generamos la columna localización POINT(latitude,longitude)
cleanOld["location"] = cleanOld[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
cleanOld = cleanOld.drop(columns=["latitude","longitude"])
display(cleanOld.head(8))
print(cleanOld.shape)

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location
0,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,"{'type': 'Point', 'coordinates': [-122.247573,..."
1,Geni,web,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,"{'type': 'Point', 'coordinates': [-118.393064,..."
2,Digg,news,None,135 Mississippi St,,94107,San Francisco,CA,USA,"{'type': 'Point', 'coordinates': [-122.394523,..."
3,Fox Interactive Media,web,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,"{'type': 'Point', 'coordinates': [-118.39417, ..."
4,Gizmoz,web,None,None,None,None,Menlo Park,CA,USA,"{'type': 'Point', 'coordinates': [-122.169472,..."
5,StumbleUpon,web,,,,,San Francisco,CA,USA,"{'type': 'Point', 'coordinates': [-122.419204,..."
5,StumbleUpon,web,,,,,New York City,NY,USA,None
6,Slacker,music,,16935 W. Bernardo Dr. Suite 101,,92127,San Diego,CA,USA,"{'type': 'Point', 'coordinates': [-117.081406,..."


(15076, 10)


In [16]:
cleanOld.country_code.value_counts()

USA    8004
GBR    1094
CAN     482
IND     442
DEU     339
       ... 
BOL       1
PST       1
KGZ       1
GHA       1
LAO       1
Name: country_code, Length: 111, dtype: int64

In [17]:
cleanOld = cleanOld[cleanOld['country_code'].isin(["USA"])]

In [18]:
cleanOld.country_code.value_counts()

USA    8004
Name: country_code, dtype: int64

In [19]:
cleanOld.shape

(8004, 10)

In [20]:
cleanOld.location.isna().value_counts()

False    5897
True     2107
Name: location, dtype: int64

In [21]:
cleanOld = cleanOld.dropna(subset=["location"])

In [22]:
cleanOld

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location
0,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,"{'type': 'Point', 'coordinates': [-122.247573,..."
1,Geni,web,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,"{'type': 'Point', 'coordinates': [-118.393064,..."
2,Digg,news,None,135 Mississippi St,,94107,San Francisco,CA,USA,"{'type': 'Point', 'coordinates': [-122.394523,..."
3,Fox Interactive Media,web,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,"{'type': 'Point', 'coordinates': [-118.39417, ..."
4,Gizmoz,web,None,None,None,None,Menlo Park,CA,USA,"{'type': 'Point', 'coordinates': [-122.169472,..."
...,...,...,...,...,...,...,...,...,...,...
12337,Axon Solutions,software,,15 Exchange Place,Suite 730,07302,Jersey City,NJ,USA,"{'type': 'Point', 'coordinates': [-74.033251, ..."
12338,Contemporary Computer Services,software,,200 Knickerbocker Avenue,,11716,Bohemia,NY,USA,"{'type': 'Point', 'coordinates': [-73.08814, 4..."
12341,QSGI,software,,400 Royal Palm Way,,33480,Palm Beach,FL,USA,"{'type': 'Point', 'coordinates': [-80.041395, ..."
12342,AfterLogic,software,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,"{'type': 'Point', 'coordinates': [-74.3235539,..."


In [23]:
cleanOld.to_json("OUTPUT/01old_offices.json", orient="records")

In [24]:
#Import to mongodb:
# $ mongoimport --db companies --collection oldcompanies --jsonArray --drop OUTPUT/01old_offices.json

## FILTER 2 | Airports

In [25]:
datair = requests.get("https://raw.githubusercontent.com/jbrooksuk/JSON-Airports/master/airports.json").json()
dfair = pd.DataFrame(datair)
display(dfair.head(8))
print(dfair.shape)

,iata,lon,iso,status,name,continent,type,lat,size
0,UTK,169.86667,MH,1,Utirik Airport,OC,airport,11.233333,small
1,FIV,NaN,US,1,Five Finger CG Heliport,NA,heliport,NaN,None
2,FAK,NaN,US,1,False Island Seaplane Base,NA,seaplanes,NaN,None
3,BWS,NaN,US,0,Blaine Municipal Airport,NA,closed,NaN,None
4,WKK,-158.61111,US,1,Aleknagik / New Airport,NA,airport,59.27778,medium
5,TSS,NaN,US,1,East 34th Street Heliport,NA,heliport,NaN,None
6,FOB,-123.79444,US,1,Fort Bragg Airport,NA,airport,39.474445,small
7,ABP,141.1,PG,1,Atkamba Airport,OC,airport,-6.066667,small


(6726, 9)


In [26]:
dfair["size"].value_counts()

medium    3556
small     2485
large      505
Name: size, dtype: int64

In [27]:
dfair = dfair[dfair['size'].isin(["large","medium"])]
dfair = dfair.dropna(subset=["size","lon","lat"])
display(dfair.head())
print(dfair.shape)

,iata,lon,iso,status,name,continent,type,lat,size
4,WKK,-158.61111,US,1,Aleknagik / New Airport,NA,airport,59.27778,medium
10,TJP,-66.563545,PR,1,Areopuerto Internacional Michael Gonzalez,NA,airport,18.010702,large
24,HIR,160.04819,SB,1,Honiara International Airport,OC,airport,-9.428592,medium
28,MUA,157.26971,SB,1,Munda Airport,OC,airport,-8.32794,medium
50,AMC,20.283333,MX,1,None,NA,airport,11.033333,large


(3969, 9)


In [28]:
dfair = dfair[dfair['iso'].isin(["US"])]
display(dfair.head())
print(dfair.shape)

,iata,lon,iso,status,name,continent,type,lat,size
4,WKK,-158.61111,US,1,Aleknagik / New Airport,NA,airport,59.27778,medium
2174,ABE,-75.434364,US,1,Lehigh Valley International Airport,NA,airport,40.651573,medium
2175,ABI,-99.679726,US,1,Abilene Regional Airport,NA,airport,32.409443,medium
2176,ABQ,-106.617195,US,1,Albuquerque International Sunport Airport,NA,airport,35.049625,large
2177,ABR,-98.42611,US,1,Aberdeen Regional Airport,NA,airport,45.4525,medium


(500, 9)


In [29]:
dfair["continent"].value_counts()

NA    500
Name: continent, dtype: int64

In [30]:
dfair = dfair.drop(["continent","type","status","iata"],axis=1)
display(dfair.head())
print(dfair.shape)

,lon,iso,name,lat,size
4,-158.61111,US,Aleknagik / New Airport,59.27778,medium
2174,-75.434364,US,Lehigh Valley International Airport,40.651573,medium
2175,-99.679726,US,Abilene Regional Airport,32.409443,medium
2176,-106.617195,US,Albuquerque International Sunport Airport,35.049625,large
2177,-98.42611,US,Aberdeen Regional Airport,45.4525,medium


(500, 5)


In [31]:
cols = ['name','iso','size','lat','lon']
dfair = dfair[cols]
display(dfair.head())
print(dfair.shape)

,name,iso,size,lat,lon
4,Aleknagik / New Airport,US,medium,59.27778,-158.61111
2174,Lehigh Valley International Airport,US,medium,40.651573,-75.434364
2175,Abilene Regional Airport,US,medium,32.409443,-99.679726
2176,Albuquerque International Sunport Airport,US,large,35.049625,-106.617195
2177,Aberdeen Regional Airport,US,medium,45.4525,-98.42611


(500, 5)


In [32]:
dfair.reset_index(drop=True,inplace=True)
display(dfair.head())
print(dfair.shape)

,name,iso,size,lat,lon
0,Aleknagik / New Airport,US,medium,59.27778,-158.61111
1,Lehigh Valley International Airport,US,medium,40.651573,-75.434364
2,Abilene Regional Airport,US,medium,32.409443,-99.679726
3,Albuquerque International Sunport Airport,US,large,35.049625,-106.617195
4,Aberdeen Regional Airport,US,medium,45.4525,-98.42611


(500, 5)


In [33]:
dfair["location"] = dfair[["lat","lon"]].apply(lambda x:asGeoJSON(x.lat,x.lon), axis=1)
display(dfair.head(8))
print(dfair.shape)

,name,iso,size,lat,lon,location
0,Aleknagik / New Airport,US,medium,59.27778,-158.61111,"{'type': 'Point', 'coordinates': [-158.61111, ..."
1,Lehigh Valley International Airport,US,medium,40.651573,-75.434364,"{'type': 'Point', 'coordinates': [-75.434364, ..."
2,Abilene Regional Airport,US,medium,32.409443,-99.679726,"{'type': 'Point', 'coordinates': [-99.679726, ..."
3,Albuquerque International Sunport Airport,US,large,35.049625,-106.617195,"{'type': 'Point', 'coordinates': [-106.617195,..."
4,Aberdeen Regional Airport,US,medium,45.4525,-98.42611,"{'type': 'Point', 'coordinates': [-98.42611, 4..."
5,Southwest Georgia Regional Airport,US,medium,31.532223,-84.19611,"{'type': 'Point', 'coordinates': [-84.19611, 3..."
6,Nantucket Memorial Airport,US,medium,41.256668,-70.05972,"{'type': 'Point', 'coordinates': [-70.05972, 4..."
7,Waco Regional Airport,US,medium,31.609133,-97.22321,"{'type': 'Point', 'coordinates': [-97.22321, 3..."


(500, 6)


In [34]:
dfair.location.isna().value_counts()

False    500
Name: location, dtype: int64

In [35]:
dfair.to_json("OUTPUT/02USA_airports.json", orient="records")

In [36]:
#Import to mongodb:
# $ mongoimport --db companies --collection usairpots --jsonArray --drop OUTPUT/02USA_airports.json

In [37]:
#Hacer que esté a menos de 10 km

## FILTER 3 | Startups

In [38]:
db, collcomp = connectMongoColl('companies','companies')
startupquery = {"$and":[{"deadpooled_year":None},
                {"founded_year":{"$gte":2000}},
                {"$where":"this.offices.length >= 1"},
                {"$where":"this.acquisitions.length >= 1"}]}

filtstartup = {"name":1,"offices":1,"category_code":1,"acquisitions":1}
startups = list(collcomp.find(startupquery,filtstartup))
dfstartups = pd.DataFrame(startups)
display(dfstartups.head())
print(dfstartups.shape)

,_id,name,category_code,acquisitions,offices
0,52cdef7c4bab8bd675297d8d,Digg,news,"[{'price_amount': None, 'price_currency_code':...","[{'description': None, 'address1': '135 Missis..."
1,52cdef7c4bab8bd675297d95,StumbleUpon,web,"[{'price_amount': 29000000, 'price_currency_co...","[{'description': '', 'address1': '', 'address2..."
2,52cdef7c4bab8bd675297d8e,Facebook,social,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'Headquarters', 'address1': '..."
3,52cdef7c4bab8bd675297d94,Twitter,social,"[{'price_amount': 15000000, 'price_currency_co...","[{'description': '', 'address1': '1355 Market ..."
4,52cdef7c4bab8bd675297da5,Technorati,advertising,"[{'price_amount': None, 'price_currency_code':...","[{'description': '', 'address1': '360 Post St...."


(588, 5)


In [39]:
# 1. Hacemos explode para generar una fila por cada elemento de la lista en la columna "offices"
dfstartups_a = dfstartups.explode('acquisitions')
display(dfstartups_a.head(6))
print(dfstartups_a.shape)


,_id,name,category_code,acquisitions,offices
0,52cdef7c4bab8bd675297d8d,Digg,news,"{'price_amount': None, 'price_currency_code': ...","[{'description': None, 'address1': '135 Missis..."
1,52cdef7c4bab8bd675297d95,StumbleUpon,web,"{'price_amount': 29000000, 'price_currency_cod...","[{'description': '', 'address1': '', 'address2..."
1,52cdef7c4bab8bd675297d95,StumbleUpon,web,"{'price_amount': None, 'price_currency_code': ...","[{'description': '', 'address1': '', 'address2..."
2,52cdef7c4bab8bd675297d8e,Facebook,social,"{'price_amount': None, 'price_currency_code': ...","[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d8e,Facebook,social,"{'price_amount': 47500000, 'price_currency_cod...","[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d8e,Facebook,social,"{'price_amount': None, 'price_currency_code': ...","[{'description': 'Headquarters', 'address1': '..."


(1213, 5)


In [40]:
#2. Hacemos expand de los datos de la columna office, para generar una culumna nueva 
#por cada valor dentro de la lista. axis=1 quiere decir que se recorre fila a fila.
dfst = dfstartups_a[["acquisitions"]].apply(lambda r: r.acquisitions, result_type="expand", axis=1)
display(dfst.head(6))
print(dfst.shape)


,price_amount,price_currency_code,term_code,source_url,source_description,acquired_year,acquired_month,acquired_day,company
0,NaN,USD,None,,,2009.0,10.0,15.0,"{'name': 'Wefollow', 'permalink': 'wefollow'}"
1,29000000.0,USD,None,http://techcrunch.com/2009/04/13/ebay-unacquir...,StumbleUpon Beats Skype In Escaping EBay's Clu...,2009.0,4.0,NaN,"{'name': 'StumbleUpon', 'permalink': 'stumbleu..."
1,NaN,USD,None,http://techcrunch.com/2013/09/24/stumbleupon-a...,"In Its First Acquisition, StumbleUpon Buys Vid...",2013.0,9.0,24.0,"{'name': '5by', 'permalink': '5by'}"
2,NaN,USD,cash,http://www.techcrunch.com/2007/07/19/breaking-...,,2007.0,7.0,1.0,"{'name': 'Parakey', 'permalink': 'parakey'}"
2,47500000.0,USD,cash_and_stock,http://www.techcrunch.com/2009/08/10/facebook-...,Facebook Acquires FriendFeed,2009.0,8.0,10.0,"{'name': 'FriendFeed', 'permalink': 'friendfeed'}"
2,NaN,USD,None,http://techcrunch.com/2010/02/19/octazen-what-...,,2010.0,2.0,19.0,"{'name': 'Octazen Solutions', 'permalink': 'oc..."


(1213, 9)


In [41]:
#3. Concatenamos con el df original y limpiamos los datos
cleanstartups = pd.concat([dfstartups_a,dfst], axis=1)
display(cleanstartups.head(6))
print(cleanstartups.shape)

,_id,name,category_code,acquisitions,offices,price_amount,price_currency_code,term_code,source_url,source_description,acquired_year,acquired_month,acquired_day,company
0,52cdef7c4bab8bd675297d8d,Digg,news,"{'price_amount': None, 'price_currency_code': ...","[{'description': None, 'address1': '135 Missis...",NaN,USD,None,,,2009.0,10.0,15.0,"{'name': 'Wefollow', 'permalink': 'wefollow'}"
1,52cdef7c4bab8bd675297d95,StumbleUpon,web,"{'price_amount': 29000000, 'price_currency_cod...","[{'description': '', 'address1': '', 'address2...",29000000.0,USD,None,http://techcrunch.com/2009/04/13/ebay-unacquir...,StumbleUpon Beats Skype In Escaping EBay's Clu...,2009.0,4.0,NaN,"{'name': 'StumbleUpon', 'permalink': 'stumbleu..."
1,52cdef7c4bab8bd675297d95,StumbleUpon,web,"{'price_amount': None, 'price_currency_code': ...","[{'description': '', 'address1': '', 'address2...",NaN,USD,None,http://techcrunch.com/2013/09/24/stumbleupon-a...,"In Its First Acquisition, StumbleUpon Buys Vid...",2013.0,9.0,24.0,"{'name': '5by', 'permalink': '5by'}"
2,52cdef7c4bab8bd675297d8e,Facebook,social,"{'price_amount': None, 'price_currency_code': ...","[{'description': 'Headquarters', 'address1': '...",NaN,USD,cash,http://www.techcrunch.com/2007/07/19/breaking-...,,2007.0,7.0,1.0,"{'name': 'Parakey', 'permalink': 'parakey'}"
2,52cdef7c4bab8bd675297d8e,Facebook,social,"{'price_amount': 47500000, 'price_currency_cod...","[{'description': 'Headquarters', 'address1': '...",47500000.0,USD,cash_and_stock,http://www.techcrunch.com/2009/08/10/facebook-...,Facebook Acquires FriendFeed,2009.0,8.0,10.0,"{'name': 'FriendFeed', 'permalink': 'friendfeed'}"
2,52cdef7c4bab8bd675297d8e,Facebook,social,"{'price_amount': None, 'price_currency_code': ...","[{'description': 'Headquarters', 'address1': '...",NaN,USD,None,http://techcrunch.com/2010/02/19/octazen-what-...,,2010.0,2.0,19.0,"{'name': 'Octazen Solutions', 'permalink': 'oc..."


(1213, 14)


In [42]:
cleanstartups = cleanstartups.drop(columns=["_id","acquisitions","term_code","source_url","source_description","company","acquired_month","acquired_day","acquired_year"])
display(cleanstartups.head(6))
print(cleanstartups.shape)

,name,category_code,offices,price_amount,price_currency_code
0,Digg,news,"[{'description': None, 'address1': '135 Missis...",NaN,USD
1,StumbleUpon,web,"[{'description': '', 'address1': '', 'address2...",29000000.0,USD
1,StumbleUpon,web,"[{'description': '', 'address1': '', 'address2...",NaN,USD
2,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",NaN,USD
2,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",47500000.0,USD
2,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",NaN,USD


(1213, 5)


In [43]:
cleanstartups.price_amount.isna().value_counts()

True     998
False    215
Name: price_amount, dtype: int64

In [44]:
cleanstartups.price_currency_code.value_counts()

USD    1186
GBP      15
EUR       8
          2
JPY       1
CAD       1
Name: price_currency_code, dtype: int64

In [45]:
cleanstartups = cleanstartups.dropna(subset=["price_amount","price_currency_code"])
display(cleanstartups.head(6))
print(cleanstartups.shape)

,name,category_code,offices,price_amount,price_currency_code
1,StumbleUpon,web,"[{'description': '', 'address1': '', 'address2...",2.900000e+07,USD
2,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",4.750000e+07,USD
2,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",2.500000e+06,USD
2,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",1.000000e+07,USD
2,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",6.000000e+07,USD
2,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",1.010470e+09,USD


(215, 5)


In [46]:
cleanstartups.reset_index()

,index,name,category_code,offices,price_amount,price_currency_code
0,1,StumbleUpon,web,"[{'description': '', 'address1': '', 'address2...",29000000.0,USD
1,2,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",47500000.0,USD
2,2,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",2500000.0,USD
3,2,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",10000000.0,USD
4,2,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",60000000.0,USD
...,...,...,...,...,...,...
210,576,Everyday Solutions,mobile,"[{'description': 'HQ', 'address1': '54 Baker A...",5000000.0,USD
211,577,InfoLogix,enterprise,"[{'description': '', 'address1': '101 East Cou...",2140000.0,USD
212,578,optionsXpress,software,"[{'description': '', 'address1': '311 West Mon...",17970000.0,USD
213,586,Block Shield,hardware,"[{'description': '', 'address1': '190 Strand',...",21350000.0,USD


In [47]:
dfprice = cleanstartups.drop(columns="offices")
dfprice = dfprice.drop_duplicates()
dfprice = dfprice.groupby("name").sum().reset_index()
dfprice

,name,price_amount
0,ACM Tech,5.000000e+04
1,Accenture,6.910000e+08
2,Adconion Media Group,2.000000e+07
3,Adknowledge,7.730000e+07
4,AdultSpace,1.000000e+06
...,...,...
114,blinkx,1.195000e+08
115,iomart Group,6.055000e+07
116,mediaFORGE,1.300000e+09
117,net-m,1.500000e+00


In [48]:
dfprice = dfprice[dfprice.price_amount >= 1000000]
dfprice

,name,price_amount
1,Accenture,6.910000e+08
2,Adconion Media Group,2.000000e+07
3,Adknowledge,7.730000e+07
4,AdultSpace,1.000000e+06
6,Amobee,2.000000e+07
...,...,...
111,Zynga,2.368000e+08
114,blinkx,1.195000e+08
115,iomart Group,6.055000e+07
116,mediaFORGE,1.300000e+09


In [49]:
dfstartups.name.value_counts()

Occipital              2
Koofers                2
Crisp Thinking         2
Where I've Been        2
Procera Networks       2
                      ..
Hitachi GST            1
Smilebox               1
Rally Software         1
Brickfish              1
InterGraphicDESIGNS    1
Name: name, Length: 558, dtype: int64

In [50]:
dfprice.price_amount [dfprice.name == 'Accenture'][1]

691000000.0

In [51]:
names=[]
for n in dfprice.name:
    names.append(n)

newcol=[]
for i in dfstartups.name:
    if i in names:
        newcol.append("YES")
        #newcol.append(dfprice.price_amount[dfprice.name == i][1])
    else:
        newcol.append("NO")

dfstartups["great1million"]=newcol
dfstartups=dfstartups.drop(columns="acquisitions")
dfstartups.head()


,_id,name,category_code,offices,great1million
0,52cdef7c4bab8bd675297d8d,Digg,news,"[{'description': None, 'address1': '135 Missis...",NO
1,52cdef7c4bab8bd675297d95,StumbleUpon,web,"[{'description': '', 'address1': '', 'address2...",YES
2,52cdef7c4bab8bd675297d8e,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",YES
3,52cdef7c4bab8bd675297d94,Twitter,social,"[{'description': '', 'address1': '1355 Market ...",YES
4,52cdef7c4bab8bd675297da5,Technorati,advertising,"[{'description': '', 'address1': '360 Post St....",NO


In [52]:
print(dfstartups.shape)
dfstartups.great1million.value_counts()

(588, 5)


NO     476
YES    112
Name: great1million, dtype: int64

In [53]:
start = dfstartups[dfstartups.great1million.isin(["YES"])]
display(start.head())
print(start.shape)

,_id,name,category_code,offices,great1million
1,52cdef7c4bab8bd675297d95,StumbleUpon,web,"[{'description': '', 'address1': '', 'address2...",YES
2,52cdef7c4bab8bd675297d8e,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",YES
3,52cdef7c4bab8bd675297d94,Twitter,social,"[{'description': '', 'address1': '1355 Market ...",YES
8,52cdef7c4bab8bd675297db5,LifeLock,security,"[{'description': '', 'address1': '60 E. Rio Sa...",YES
9,52cdef7c4bab8bd675297dbc,Skype,software,"[{'description': 'Global HQ', 'address1': '23-...",YES


(112, 5)


In [54]:
#1. Hacemos explode para generar una fila por cada elemento de la lista en la columna "offices"
start = start.explode('offices')
#display(start.head(6))
#print(start.shape)

#2. Hacemos expand de los datos de la columna office, para generar una culumna nueva 
#por cada valor dentro de la lista. axis=1 quiere decir que se recorre fila a fila.
dfstart = start[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
#display(dfstart.head(6))
#print(dfstart.shape)

#3. Concatenamos con el df original y limpiamos los datos
cleanstart = pd.concat([start,dfstart], axis=1)
cleanstart = cleanstart.drop(columns=["_id","offices"])
#display(cleanstart.head(6))
#print(cleanstart.shape)

#4. Usando la funciónn asGeoJSON: Generamos la columna localización POINT(latitude,longitude)
cleanstart["location"] = cleanstart[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
#cleanstart = cleanstart.drop(columns=["latitude","longitude"])
#display(cleanstart.head(8))
#print(cleanstart.shape)

In [55]:
cleanstart.location.isna().value_counts()

False    122
True     103
Name: location, dtype: int64

In [56]:
cleanstart = cleanstart.dropna(subset=["location"])
cleanstart.head()

,name,category_code,great1million,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,location
1,StumbleUpon,web,YES,,,,,San Francisco,CA,USA,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,..."
2,Facebook,social,YES,Headquarters,1601 Willow Road,,94025,Menlo Park,CA,USA,37.416050,-122.151801,"{'type': 'Point', 'coordinates': [-122.151801,..."
2,Facebook,social,YES,Europe HQ,,,,Dublin,None,IRL,53.344104,-6.267494,"{'type': 'Point', 'coordinates': [-6.267494, 5..."
2,Facebook,social,YES,New York,340 Madison Ave,,10017,New York,NY,USA,40.755716,-73.979247,"{'type': 'Point', 'coordinates': [-73.9792469,..."
3,Twitter,social,YES,,1355 Market St.,,94103,San Francisco,CA,USA,37.776805,-122.416924,"{'type': 'Point', 'coordinates': [-122.4169244..."


In [57]:
cleanstart.location.isna().value_counts()

False    122
Name: location, dtype: int64

In [58]:
cleanstart.country_code.value_counts()

USA    89
GBR    11
CAN     5
DEU     4
IND     3
SWE     2
FRA     2
NLD     2
LUX     1
IRL     1
CHN     1
ITA     1
Name: country_code, dtype: int64

In [59]:
cleanstart = cleanstart[cleanstart.country_code.isin(["USA"])]
display(cleanstart.head())
print(cleanstart.shape)

,name,category_code,great1million,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,location
1,StumbleUpon,web,YES,,,,,San Francisco,CA,USA,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,..."
2,Facebook,social,YES,Headquarters,1601 Willow Road,,94025,Menlo Park,CA,USA,37.416050,-122.151801,"{'type': 'Point', 'coordinates': [-122.151801,..."
2,Facebook,social,YES,New York,340 Madison Ave,,10017,New York,NY,USA,40.755716,-73.979247,"{'type': 'Point', 'coordinates': [-73.9792469,..."
3,Twitter,social,YES,,1355 Market St.,,94103,San Francisco,CA,USA,37.776805,-122.416924,"{'type': 'Point', 'coordinates': [-122.4169244..."
8,LifeLock,security,YES,,"60 E. Rio Salado Pkwy, Suite 400",,85281,Tempe,AZ,USA,33.429859,-111.938739,"{'type': 'Point', 'coordinates': [-111.938739,..."


(89, 13)


In [60]:
cleanstart.to_json("OUTPUT/03USA_startups.json", orient="records")

In [61]:
#Import to mongodb:
# $ mongoimport --db companies --collection usastartups --jsonArray --drop OUTPUT/03USA_startups.json

In [62]:
# HACER QUE ESTÉN A MENOS DE 40 KM.

## FILTER 4 | Near Design companies

In [63]:
db, collcomp = connectMongoColl('companies','companies')
designquery = {"$and":[{"deadpooled_year":None},
                {"$where":"this.offices.length >= 1"},
                {"$where":"this.acquisitions.length >= 1"}]}

filtdesign = {"name":1,"offices":1,"category_code":1}
design = list(collcomp.find(designquery,filtdesign))
dfd = pd.DataFrame(design)
display(dfd.head())
print(dfd.shape)

,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8d,Digg,news,"[{'description': None, 'address1': '135 Missis..."
1,52cdef7c4bab8bd675297d93,Fox Interactive Media,web,"[{'description': '', 'address1': '407 N Maple ..."
2,52cdef7c4bab8bd675297d95,StumbleUpon,web,"[{'description': '', 'address1': '', 'address2..."
3,52cdef7c4bab8bd675297d9e,CBS,news,"[{'description': None, 'address1': '', 'addres..."
4,52cdef7c4bab8bd675297d8e,Facebook,social,"[{'description': 'Headquarters', 'address1': '..."


(1359, 4)


In [64]:
dfd.category_code.value_counts()

software            246
web                 186
enterprise          111
mobile               86
advertising          86
games_video          70
hardware             62
other                56
network_hosting      53
ecommerce            39
public_relations     35
security             35
consulting           35
biotech              31
semiconductor        27
cleantech            22
news                 18
social               18
search               14
finance               7
travel                6
education             5
analytics             5
transportation        5
manufacturing         4
messaging             4
real_estate           4
music                 4
hospitality           3
automotive            2
photo_video           2
legal                 2
fashion               2
health                2
design                1
nonprofit             1
sports                1
Name: category_code, dtype: int64

In [65]:
design_param = ["software","web","design","music","analytics","biotech","cleantech","consulting","ecommerce","games_video","network_hosting"]
dfd = dfd[dfd.category_code.isin(design_param)]
dfd.head()


,_id,name,category_code,offices
1,52cdef7c4bab8bd675297d93,Fox Interactive Media,web,"[{'description': '', 'address1': '407 N Maple ..."
2,52cdef7c4bab8bd675297d95,StumbleUpon,web,"[{'description': '', 'address1': '', 'address2..."
5,52cdef7c4bab8bd675297d9f,Viacom,web,"[{'description': 'HQ', 'address1': '1515 Broad..."
7,52cdef7c4bab8bd675297d9b,eBay,web,"[{'description': 'Headquarters', 'address1': '..."
11,52cdef7c4bab8bd675297dad,Gannett,web,"[{'description': None, 'address1': '7950 Jones..."


In [66]:
#1. Hacemos explode para generar una fila por cada elemento de la lista en la columna "offices"
dfd = dfd.explode('offices')
#display(dfd.head(6))
#print(dfd.shape)

#2. Hacemos expand de los datos de la columna office, para generar una culumna nueva 
#por cada valor dentro de la lista. axis=1 quiere decir que se recorre fila a fila.
dfdexpand = dfd[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
#display(dfdexpand.head(6))
#print(dfdexpand.shape)

#3. Concatenamos con el df original y limpiamos los datos
cleandesign = pd.concat([dfd,dfdexpand], axis=1)
cleandesign = cleandesign.drop(columns=["_id","offices"])
#display(cleandesign.head(6))
#print(cleandesign.shape)

#4. Usando la funciónn asGeoJSON: Generamos la columna localización POINT(latitude,longitude)
cleandesign["location"] = cleandesign[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
#cleandesign = cleanstart.drop(columns=["latitude","longitude"])
#display(cleandesign.head(8))
#print(cleandesign.shape)


In [67]:
cleandesign.location.isna().value_counts()

False    620
True     397
Name: location, dtype: int64

In [68]:
cleandesign = cleandesign.dropna(subset=["location"])
cleandesign.head()

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,location
1,Fox Interactive Media,web,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,34.076179,-118.394170,"{'type': 'Point', 'coordinates': [-118.39417, ..."
2,StumbleUpon,web,,,,,San Francisco,CA,USA,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,..."
5,Viacom,web,HQ,1515 Broadway,,10036,New York,NY,USA,40.757725,-73.986011,"{'type': 'Point', 'coordinates': [-73.986011, ..."
7,eBay,web,Headquarters,2145 Hamilton Avenue,,95125,San Jose,CA,USA,37.295005,-121.930035,"{'type': 'Point', 'coordinates': [-121.930035,..."
11,Gannett,web,None,7950 Jones Branch Drive,,22107,McLean,VA,USA,38.929289,-77.216585,"{'type': 'Point', 'coordinates': [-77.216585, ..."


In [69]:
cleandesign.country_code.value_counts()


USA    492
GBR     33
CAN     19
DEU     14
FRA     10
AUS      7
ARG      6
SWE      6
NLD      5
BEL      3
ITA      3
IND      3
LUX      2
ESP      2
BRA      2
CHN      2
CHE      2
COL      1
POL      1
JPN      1
URY      1
DNK      1
MYS      1
NOR      1
ARE      1
SGP      1
Name: country_code, dtype: int64

In [70]:
cleandesign = cleandesign[cleandesign.country_code.isin(["USA"])]
display(cleandesign.head())
print(cleandesign.shape)


,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,location
1,Fox Interactive Media,web,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,34.076179,-118.394170,"{'type': 'Point', 'coordinates': [-118.39417, ..."
2,StumbleUpon,web,,,,,San Francisco,CA,USA,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,..."
5,Viacom,web,HQ,1515 Broadway,,10036,New York,NY,USA,40.757725,-73.986011,"{'type': 'Point', 'coordinates': [-73.986011, ..."
7,eBay,web,Headquarters,2145 Hamilton Avenue,,95125,San Jose,CA,USA,37.295005,-121.930035,"{'type': 'Point', 'coordinates': [-121.930035,..."
11,Gannett,web,None,7950 Jones Branch Drive,,22107,McLean,VA,USA,38.929289,-77.216585,"{'type': 'Point', 'coordinates': [-77.216585, ..."


(492, 12)


In [71]:
cleandesign.to_json("OUTPUT/04USA_design.json", orient="records")

In [72]:
#Import to mongodb:
# $ mongoimport --db companies --collection usadesign --jsonArray --drop OUTPUT/04USA_design.json

In [73]:
#Hacer que estén a menos de 10 km

## FINAL MERGE: APPLY FILTERS
---

In [74]:
def getValue(df1,df2,compare):
    lst=[]
    for i in df1.location:
        a=0

        for elem in df2.location:
            if haversine(i["coordinates"],elem["coordinates"]) < int(compare):
                a+=1
        lst.append(a)

    return lst

### FILTRO 1 - Old companies

df obtenido = cleanData

In [75]:
lst_f1=getValue(cleanData,cleanOld,2)
print(len(lst_f1))

7600


In [76]:
cleanData.loc[:, 'f_oldcomp'] = lst_f1

In [77]:
cleanData.head()

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location,f_oldcomp
0,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,...",10
1,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,"{'type': 'Point', 'coordinates': [-122.333253,...",89
1,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,"{'type': 'Point', 'coordinates': [-73.9964312,...",406
2,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,...",10
4,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,"{'type': 'Point', 'coordinates': [-122.247573,...",30


In [78]:
dfilt1 = cleanData.sort_values('f_oldcomp').reset_index(drop=True)
dfilt1 = dfilt1[dfilt1["f_oldcomp"]<=5]
dfilt1.shape

(2741, 11)

In [79]:
dfilt1.to_json("OUTPUT/FILTER1_Oldcomp_2741.json", orient="records")

In [80]:
center = [ -100.619746, 40.187280]
lstbool = []
for i in dfilt1.location:
    if haversine(i['coordinates'],center) > 2500:
        lstbool.append("OUT")
    else:
        lstbool.append("IN")
print(len(lstbool))

2741


In [81]:
dfilt1.loc[:, "in_USA"] = lstbool

In [82]:
dfilt1.head()

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location,f_oldcomp,in_USA
0,Veritas Prep,other,Headquarters,22741 Pacific Coast Highway,Suite 300,90265,Malibu,CA,USA,"{'type': 'Point', 'coordinates': [-118.6707158...",0,IN
1,Intellon Corporation,semiconductor,,5100 West Silver Springs Blvd.,,34482,Ocala,FL,USA,"{'type': 'Point', 'coordinates': [-82.2046985,...",0,IN
2,XunLight,cleantech,None,3145 Nebraska Avenue,,43607,Toledo,OH,USA,"{'type': 'Point', 'coordinates': [-83.618593, ...",0,IN
3,Borrego Solar Systems,cleantech,San Diego Location,"1810 Gillespie Way, Suite 108",,92020,El Cajon,CA,USA,"{'type': 'Point', 'coordinates': [-116.981232,...",0,IN
4,Solicore,other,,2700 Interstate Drive,,33805,Lakeland,FL,USA,"{'type': 'Point', 'coordinates': [-81.9752196,...",0,IN


In [83]:
dfilt1.in_USA.value_counts()

IN     1992
OUT     749
Name: in_USA, dtype: int64

In [84]:
dffin = dfilt1[dfilt1.in_USA.isin(["IN"])]

In [85]:
dffin.shape

(1992, 12)

In [86]:
dffin.to_json("OUTPUT/FILTER1_1_Oldcomp_1992.json", orient="records")

In [87]:
# $ mongoimport --db companies --collection filter1_1_oldcomp --jsonArray --drop OUTPUT/FILTER1_1_Oldcomp_1992.json


### FILTRO 2 - Airports

df obtenido = dfair

In [88]:
dfilt2 = dffin.drop(columns=["category_code",'description','address1','address2','zip_code','country_code','in_USA'])

In [89]:
lst_air=getValue(dfilt2,dfair,5)
print(len(lst_air))

1992


In [90]:
dfilt2.loc[:, "near_airport"] = lst_air
dfilt2 = dfilt2[dfilt2["near_airport"]>=1]

In [91]:
dfilt2.shape

(417, 6)

In [92]:
dfilt2.reset_index(drop=True).head()

,name,city,state_code,location,f_oldcomp,near_airport
0,Mortgagebot,Mequon,WI,"{'type': 'Point', 'coordinates': [-87.9218939,...",0,1
1,Dolphin Geeks,,FL,"{'type': 'Point', 'coordinates': [-80.421525, ...",0,1
2,Adaptive Ozone Solutions,Lenexa,KS,"{'type': 'Point', 'coordinates': [-94.715428, ...",0,1
3,Extreme Enterprises,St Petersburg,FL,"{'type': 'Point', 'coordinates': [-82.735899, ...",0,1
4,VasoGenix,Lenexa,KS,"{'type': 'Point', 'coordinates': [-94.710676, ...",0,1


In [93]:
dfilt2.to_json("OUTPUT/FILTER2_Airport_417.json", orient="records")
# $ mongoimport --db companies --collection filt2_airport --jsonArray --drop OUTPUT/FILTER2_Airport_417.json

### FILTRO 3 - Startups

df obtenido = cleanstart

In [94]:
dfilt3 = dfilt2

In [95]:
lst_startups=getValue(dfilt3,cleanstart,40)
print(len(lst_startups))

417


In [96]:
dfilt3.loc[:, "near_startup"] = lst_startups
dfilt3 = dfilt3[dfilt3["near_startup"]>1]

In [97]:
dfilt3.head()

,name,city,state_code,location,f_oldcomp,near_airport,near_startup
14,Mortgagebot,Mequon,WI,"{'type': 'Point', 'coordinates': [-87.9218939,...",0,1,4
48,Extreme Enterprises,St Petersburg,FL,"{'type': 'Point', 'coordinates': [-82.735899, ...",0,1,2
125,SDH Group,St. Petersburg,FL,"{'type': 'Point', 'coordinates': [-82.663026, ...",0,1,2
166,Rockwell Automation,Milwaukee,WI,"{'type': 'Point', 'coordinates': [-87.91334, 4...",0,2,4
269,Scary Ideas,Bradenton,FL,"{'type': 'Point', 'coordinates': [-82.580968, ...",1,1,2


In [98]:
dfilt3.to_json("OUTPUT/FILTER3_Startups_126.json", orient="records")
# $ mongoimport --db companies --collection filt3_startups --jsonArray --drop OUTPUT/FILTER3_Startups_126.json

### FILTRO 4 - Design companies

df obtenido = cleandesign

In [99]:
dfilt4 = dfilt3

In [100]:
lstdesign = getValue(dfilt4,cleandesign,10)

In [101]:
dfilt4.loc[:, "near_design"] = lstdesign
dfilt4 = dfilt4[dfilt4["near_design"]>1]

/Users/miguelceinos/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/miguelceinos/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [102]:
dfilt4.head()

,name,city,state_code,location,f_oldcomp,near_airport,near_startup,near_design
14,Mortgagebot,Mequon,WI,"{'type': 'Point', 'coordinates': [-87.9218939,...",0,1,4,3
166,Rockwell Automation,Milwaukee,WI,"{'type': 'Point', 'coordinates': [-87.91334, 4...",0,2,4,3
307,SAS,Cary,NC,"{'type': 'Point', 'coordinates': [-78.765676, ...",1,1,3,3
319,Emagic,Milwaukee,WI,"{'type': 'Point', 'coordinates': [-87.909151, ...",1,2,4,3
335,GMA Grading,Odessa,FL,"{'type': 'Point', 'coordinates': [-82.568494, ...",1,1,2,2


In [103]:
dfilt4.shape

(82, 8)

In [104]:
dfilt4.to_json("OUTPUT/FILTER4_Design_82.json", orient="records")
# $ mongoimport --db companies --collection filt4_Design --jsonArray --drop OUTPUT/FILTER4_Design_82.json

# TOP20

In [105]:
dfinal = dfilt4.sort_values(by="f_oldcomp",ascending=True).sort_values(by=['near_startup','near_design','near_airport'],ascending=False)
dfinal = dfinal.head(20).reset_index(drop=True)
dfinal

,name,city,state_code,location,f_oldcomp,near_airport,near_startup,near_design
0,Shopseen,San Francisco,CA,"{'type': 'Point', 'coordinates': [-122.214897,...",2,1,27,3
1,Nibea,Oakland,CA,"{'type': 'Point', 'coordinates': [-122.214897,...",2,1,27,3
2,Handipoints,Oakland,CA,"{'type': 'Point', 'coordinates': [-122.214897,...",2,1,27,3
3,Singulex,Alameda,CA,"{'type': 'Point', 'coordinates': [-122.240937,...",2,1,26,4
4,PINC Solutions,Alameda,CA,"{'type': 'Point', 'coordinates': [-122.243783,...",2,1,26,4
5,Qlubb,Burlingame,CA,"{'type': 'Point', 'coordinates': [-122.369448,...",4,1,22,11
6,Sky Bird Travel Tours,Burlingame,CA,"{'type': 'Point', 'coordinates': [-122.368281,...",4,1,22,11
7,ScanCafe,Burlingame,CA,"{'type': 'Point', 'coordinates': [-122.369676,...",4,1,22,10
8,Gen-Y Media,Burlingame,CA,"{'type': 'Point', 'coordinates': [-122.376572,...",4,1,21,10
9,XDx,Brisbane,CA,"{'type': 'Point', 'coordinates': [-122.4040269...",3,1,18,51


In [106]:
dfinal.to_json("OUTPUT/TOP20.json", orient="records")
# $ mongoimport --db companies --collection TOP20 --jsonArray --drop OUTPUT/TOP20.json

In [107]:
for ind,elem in enumerate(dfinal.location):
    print(ind,elem["coordinates"])

0 [-122.214897, 37.772323]
1 [-122.214897, 37.772323]
2 [-122.214897, 37.772323]
3 [-122.240937, 37.723803]
4 [-122.243783, 37.726339]
5 [-122.369448, 37.567832]
6 [-122.368281, 37.599131]
7 [-122.369676, 37.592584]
8 [-122.376572, 37.598478]
9 [-122.4040269, 37.691226]
10 [-122.42689, 37.636288]
11 [-122.422113, 37.630551]
12 [-122.424975, 37.629393]
13 [-122.422562, 37.62815]
14 [-122.422833, 37.628933]
15 [-122.422833, 37.628933]
16 [-122.420814, 37.628328]
17 [-122.424607, 37.628518]
18 [-122.426804, 37.627971]
19 [-121.911101, 37.3190224]
